<a href="https://colab.research.google.com/github/seamusrobertmurphy/01-data-processing/blob/main/01-data-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment setup

In [ ]:
!pip install leafmap geemap==0.16.4 geopandas numpy session_info
import ee, json, geemap, ipyleaflet, os, numpy, backports, session_info
from google.colab import drive
from google.colab import files
!drive.mount('/content/drive')

### Activate Earth Engine

In [ ]:
#!ee.Authenticate() # deprecated in certain Colab environments
!earthengine authenticate
ee.Initialize(project = "murphys-deforisk")

### Jurisidictional boundaries

In [4]:
country = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Liberia"))
state = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Liberia"))
state_list = country.aggregate_array('ADM1_NAME').distinct().getInfo()
state_list

['Grand Gedeh',
 'River Gee',
 'Gbarpolu',
 'Lofa',
 'Bomi',
 'Bong',
 'Grand Bassa',
 'Grand Cape Mount',
 'Grand Kru',
 'Margibi',
 'Maryland',
 'Montserrado',
 'Nimba',
 'Rivercess',
 'Sinoe']

In [5]:
red = {"color": "red", "width": 2, "lineType": "solid", "fillColor": "00000000"}
white = {"color": "white", "width": 1, "lineType": "solid", "fillColor": "00000000"}
country_label = ee.FeatureCollection([ee.Feature(
    country.geometry().centroid(), {'country_name': country.first().get("ADM0_NAME").getInfo()})])

Map = geemap.Map()
Map.centerObject(country, 6)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(country.style(**white), {}, "Country")
Map.addLayer(state.style(**red), {}, "States")
Map.add_labels(state,"ADM1_NAME",font_size="9pt",font_color="red",font_family="arial",font_weight="bold",)
Map.add_labels(country_label, "country_name", font_size="12pt", font_color="white", font_family="arial",)
Map

Map(center=[6.446116651575274, -9.30729708708129], controls=(WidgetControl(options=['position', 'transparent_b…

The ImageCollection Landsat 8 Surface Reflectance Tier 1 have been atmospherically corrected using LaSRC and includes a cloud, shadow, water and snow mask produced using CFMASK, as well as a per-pixel saturation mask.

In [6]:
# derive masking, scaling, and ndvi function
def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI').toFloat()
    image = image.addBands(opticalBands, None, True) \
                 .addBands(thermalBands, None, True) \
                 .addBands(ndvi)
    return image.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'NDVI'],
        ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']
    ).updateMask(qaMask).updateMask(saturationMask)

# create collections for 2014 and 2024
collection_2014 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                    .filterDate('2014-01-01', '2014-12-31') \
                    .filterBounds(state) \
                    .map(maskL8sr)

collection_2019 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                    .filterDate('2019-01-01', '2019-12-31') \
                    .filterBounds(state) \
                    .map(maskL8sr)

collection_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                    .filterDate('2024-01-01', '2024-12-31') \
                    .filterBounds(state) \
                    .map(maskL8sr)

# median composites for 2014 and 2024
composite_2014 = collection_2014.select(['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']).median().clip(state).toFloat()
composite_2019 = collection_2019.select(['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']).median().clip(state).toFloat()
composite_2024 = collection_2024.select(['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']).median().clip(state).toFloat()


# visualization
ndviVis = {'min': 0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}
rgbVis = {'bands': ['RED', 'GREEN', 'BLUE'],'min': 0, 'max': 0.3, 'gamma': 1.4}
Map = geemap.Map()
Map.centerObject(state, 8)
Map.addLayer(composite_2014.select('NDVI'), ndviVis, 'NDVI 2014')
Map.addLayer(composite_2019.select('NDVI'), ndviVis, 'NDVI 2019')
Map.addLayer(composite_2024.select('NDVI'), ndviVis, 'NDVI 2024')
Map.addLayer(composite_2014.select(['RED', 'GREEN', 'BLUE']), rgbVis, 'RGB 2014')
Map.addLayer(composite_2019.select(['RED', 'GREEN', 'BLUE']), rgbVis, 'RGB 2019')
Map.addLayer(composite_2024.select(['RED', 'GREEN', 'BLUE']), rgbVis, 'RGB 2024')
Map.addLayer(state, {}, 'Area of Interest')
Map.addLayerControl()
Map

Map(center=[6.446116651575274, -9.30729708708129], controls=(WidgetControl(options=['position', 'transparent_b…

In [7]:
# confirm dates, scene IDs, band names of images
firstImage_2014 = collection_2014.first()
sceneId_2014 = firstImage_2014.get('system:index').getInfo()
print(f"Scene ID for collection_2014: {sceneId_2014}")

firstImage_2019 = collection_2019.first()
sceneId_2019 = firstImage_2019.get('system:index').getInfo()
print(f"Scene ID for collection_2019: {sceneId_2019}")

firstImage_2024 = collection_2024.first()
sceneId_2024 = firstImage_2024.get('system:index').getInfo()
print(f"Scene ID for collection_2024: {sceneId_2024}")

bandNames_2014 = composite_2014.bandNames().getInfo()
print(f"Band names: {bandNames_2014}")

bandNames_2019 = composite_2019.bandNames().getInfo()
print(f"Band names: {bandNames_2019}")

bandNames_2024 = composite_2024.bandNames().getInfo()
print(f"Band names: {bandNames_2024}")

Scene ID for collection_2014: LC08_198055_20140104
Scene ID for collection_2019: LC08_198055_20190102
Scene ID for collection_2024: LC08_198055_20240116
Band names: ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']
Band names: ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']
Band names: ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']


### Export to Drive

Caution, running the following chunk will export 12GB of duplicate rasters to a drive folder.

In [ ]:
from datetime import datetime

# extract pathrow and date from scene ID
def get_pathrow_date(image_collection):
  first_image = image_collection.first()
  scene_id = first_image.get('system:index').getInfo()
  parts = scene_id.split('_')
  pathrow = parts[1]
  date_str = parts[2]
  date_obj = datetime.strptime(date_str, '%Y%m%d')
  date = date_obj.strftime('%Y-%m-%d')
  return pathrow, date

# define export parameters
def define_export_params(image, pathrow, date, band_name):
  description = f'composite_{date}_{band_name if isinstance(band_name, str) else "RGB"}_export'[:100]
  return {
    'image': image.select(band_name),
    'description': description,
    'folder': 'VT0007-deforestation-map',
    'fileNamePrefix': f'LANDSAT_TM-ETM-OLI_{pathrow}_{band_name if isinstance(band_name, str) else "RGB"}_{date}',
    'scale': 30,
    'region': state.geometry(),
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
  }

# get pathrow and date for each collection
pathrow_2014, date_2014 = get_pathrow_date(collection_2014)
pathrow_2019, date_2019 = get_pathrow_date(collection_2019)
pathrow_2024, date_2024 = get_pathrow_date(collection_2024)

# get band names
bandNames_2014 = composite_2014.bandNames().getInfo()
bandNames_2019 = composite_2019.bandNames().getInfo()
bandNames_2024 = composite_2024.bandNames().getInfo()

# export to cloud bucket
for band_name in bandNames_2014:
    export_params_2014 = define_export_params(composite_2014, pathrow_2014, date_2014, band_name)
    task_2014 = ee.batch.Export.image.toDrive(**export_params_2014)
    task_2014.start()
    print(f"Exporting 2014 image for band {band_name}. Task ID: {task_2014.id}")

for band_name in bandNames_2019:
    export_params_2019 = define_export_params(composite_2019, pathrow_2019, date_2019, band_name)
    task_2019 = ee.batch.Export.image.toDrive(**export_params_2019)
    task_2019.start()
    print(f"Exporting 2019 image for band {band_name}. Task ID: {task_2019.id}")

for band_name in bandNames_2024:
    export_params_2024 = define_export_params(composite_2024, pathrow_2024, date_2024, band_name)
    task_2024 = ee.batch.Export.image.toDrive(**export_params_2024)
    task_2024.start()
    print(f"Exporting 2024 image for band {band_name}. Task ID: {task_2024.id}")

export_params_2014_rgb = define_export_params(composite_2014, pathrow_2014, date_2014, ['RED', 'GREEN', 'BLUE'])
export_params_2014_rgb['image'] = composite_2014.visualize(**rgbVis)
task_2014_rgb = ee.batch.Export.image.toDrive(**export_params_2014_rgb)
task_2014_rgb.start()
print(f"Exporting 2014 RGB image. Task ID: {task_2014_rgb.id}")

export_params_2019_rgb = define_export_params(composite_2019, pathrow_2019, date_2019, ['RED', 'GREEN', 'BLUE'])
export_params_2019_rgb['image'] = composite_2019.visualize(**rgbVis)
task_2019_rgb = ee.batch.Export.image.toDrive(**export_params_2019_rgb)
task_2019_rgb.start()
print(f"Exporting 2019 RGB image. Task ID: {task_2019_rgb.id}")

export_params_2024_rgb['image'] = composite_2024.visualize(**rgbVis)
task_2024_rgb = ee.batch.Export.image.toDrive(**export_params_2024_rgb)
task_2024_rgb.start()
print(f"Exporting 2024 RGB image. Task ID: {task_2024_rgb.id}")

Exporting 2014 RGB image. Task ID: IIMNP6IZ4ALUYMXEILXGCRND


In [13]:
session_info.show()